# 4.14. V-мера. Практика

In [2]:
#подготовка данных
from sklearn import datasets
import numpy as np
def prepare_blobs():
    n_samples = 1500
    dataset = datasets.make_blobs(n_samples=n_samples, centers=2, center_box=(-7.0, 7.5), cluster_std=[1.4, 1.7], random_state=42)
    X_2, _ = datasets.make_blobs(n_samples=n_samples, random_state=170, centers=[[-4, -3]], cluster_std=[1.9])
    transformation = [[1.2, -0.8], [-0.4, 1.7]]
    X_2 = np.dot(X_2, transformation)
    X, y = np.concatenate((dataset[0], X_2)), np.concatenate((dataset[1], np.array([2] * len(X_2))))
    return X, y

X, y = prepare_blobs()

In [3]:
from sklearn.cluster import KMeans
from sklearn.metrics.cluster import v_measure_score

# сначала получим предсказанные кластеры при помощи метода кластеризации
kmeans = KMeans(n_clusters=3, random_state=42)
kmeans.fit(X)
kmeans_pred = kmeans.labels_

# теперь посчитаем полноту
v_measure_score(labels_true=y, labels_pred=kmeans_pred)

0.795592702456356

## Задание 4.14.1

2.0/2.0 points (graded)
Обучите модель GaussianMixture с параметрами n_components=3 и random_state=42 на признаках исходного датасета. Посчитайте v-меру для получившейся кластеризации.

Подсказка: При решении задания стандартизация должна быть включена. Ответ округлите до сотых и запишите с точкой.

In [2]:
#подготовка данных
from sklearn import datasets
import numpy as np
def prepare_blobs():
    n_samples = 1500
    dataset = datasets.make_blobs(n_samples=n_samples, centers=2, center_box=(-7.0, 7.5), cluster_std=[1.4, 1.7], random_state=42)
    X_2, _ = datasets.make_blobs(n_samples=n_samples, random_state=170, centers=[[-4, -3]], cluster_std=[1.9])
    transformation = [[1.2, -0.8], [-0.4, 1.7]]
    X_2 = np.dot(X_2, transformation)
    X, y = np.concatenate((dataset[0], X_2)), np.concatenate((dataset[1], np.array([2] * len(X_2))))
    return X, y

X, y = prepare_blobs()

In [46]:
#стандартизация
from sklearn.preprocessing import StandardScaler
X_st = StandardScaler().fit_transform(X)

In [16]:
#создаем класс для всех моделей

from sklearn.cluster import DBSCAN
from sklearn.cluster import AgglomerativeClustering
from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans

class complex_fit():
    def __init__(self):
        pass
    
    def y_pred_dbscan(self, n_clusters=3, X=X ):
        dbscan = DBSCAN(eps=0.9, min_samples=35)
        dbscan.fit(X)
        y_pred_dbscan = dbscan.labels_
        return y_pred_dbscan
    
    def y_pred_ac(self, n_clusters=3, X=X):
        ac = AgglomerativeClustering(n_clusters=n_clusters)
        ac.fit(X)
        y_pred_ac = ac.labels_.astype(np.int)
        return y_pred_ac
    
    def y_pred_gm(self, n_clusters=3, X=X):
        gm = GaussianMixture(n_components=n_clusters, random_state=42)
        y_pred_gm = gm.fit_predict(X)
        return y_pred_gm
    
    def y_pred_km(self, n_clusters=3, X=X):
        k_means = KMeans(n_clusters=n_clusters, random_state=42)
        k_means.fit(X)
        #k_means.cluster_centers_
        y_pred_km = k_means.labels_
        return y_pred_km

    def pred_array(self, n_clusters=3, X=X):
        x = [self.y_pred_km(n_clusters, X=X), 
        self.y_pred_gm(n_clusters, X=X), 
        self.y_pred_ac(n_clusters, X=X), 
        self.y_pred_dbscan(n_clusters, X=X)]
        return x

In [17]:
cf = complex_fit()
pred_array = cf.pred_array(n_clusters=3, X=X)

In [20]:
for n, pred in enumerate(pred_array):
    x = v_measure_score(labels_true=y, labels_pred=pred)
    print(n+1, x)

1 0.795592702456356
2 0.9332760595996924
3 0.703747024360433
4 0.7732549110297919


### ответ 0,93

## Задание 4.14.2

2.0/2.0 points (graded)
Сравните результаты кластеризации алгоритмов K-means, GaussianMixture, AgglomerativeClustering и DBSCAN на исходном датасете при помощи v-меры, инициализируйте алгоритмы со следующими параметрами:

K-means – n_clusters=3, random_state=42
GaussianMixture – n_components=3, random_state=42
AgglomerativeClustering – n_clusters=3
DBSCAN – eps=0.9, min_samples=35
В качестве ответа укажите максимальное значение v-меры, полученное при помощи данных моделей. Ответ округлите до сотых и запишите с точкой.

In [21]:
for n, pred in enumerate(pred_array):
    x = v_measure_score(labels_true=y, labels_pred=pred)
    print(n+1, x)

1 0.795592702456356
2 0.9332760595996924
3 0.703747024360433
4 0.7732549110297919


### ответ 0,93

## Задание 4.14.3

2.0/2.0 points (graded)
Сравним модификации K-means с использованием случайной инициализации центроидов и с использованием алгоритма K-means++ для инициализации центроидов.

Для этого обучим на исходном датасете 2 модели K-means со следующими параметрами:

n_clusters=3, init='k-means++', n_init=1, random_state=42
n_clusters=3, init='random', n_init=1, random_state=42
В качестве ответа укажите максимальное значение v-меры, полученное при помощи данных моделей.

Подсказка: При решении задания стандартизация должна быть включена. Ответ округлите до сотых и запишите с точкой.

In [22]:
k_means1 = KMeans(n_clusters=3, init='k-means++', n_init=1, random_state=42)
k_means1.fit(X)
#k_means.cluster_centers_
y_pred_km1 = k_means1.labels_
print(v_measure_score(labels_true=y, labels_pred=y_pred_km1))

k_means2 = KMeans(n_clusters=3, init='random', n_init=1, random_state=42)
k_means2.fit(X)
#k_means.cluster_centers_
y_pred_km2 = k_means2.labels_
v_measure_score(labels_true=y, labels_pred=y_pred_km2)

0.7911556191133551


0.5990620007488798

### ответ 0,79

## Задание 4.14.4

2.0/2.0 points (graded)
Теперь сравним K-means с ещё одной модификацией — K-means mini batch. Воспользоваться реализацией K-means mini batch в библиотеке sklearn можно следующим образом:


k-means с параметрами n_clusters=3, n_init=1, random_state=42
MiniBatchKMeans с параметрами n_clusters=3, n_init=1, random_state=42
В качестве ответа введите максимальное значение v-меры, полученное при помощи данных моделей. В задании может понадобиться или не понадобиться нормализация — это нужно проверить во время решения задания. Ответ округлите до десятых и запишите с точкой.

In [26]:
from sklearn.cluster import MiniBatchKMeans

kmeans_mini_batch = MiniBatchKMeans(n_clusters=3, n_init=1, random_state=42)
kmeans_mini_batch.fit(X_st)
kmeans_mini_batch_pred = kmeans_mini_batch.labels_
print(v_measure_score(labels_true=y, labels_pred=kmeans_mini_batch_pred))

0.804697430165486


In [27]:
k_means1 = KMeans(n_clusters=3, n_init=1, random_state=42)
k_means1.fit(X_st)
#k_means.cluster_centers_
y_pred_km1 = k_means1.labels_
print(v_measure_score(labels_true=y, labels_pred=y_pred_km1))

0.7892280957870129


### ответ 0,8 (с нормализацией)

## Задание 4.14.5

2.0/2.0 points (graded)
Рассмотрим агломеративную кластеризацию. Сравним, как влияет на качество кластеризации разный тип расстояния между кластерами.

Обучите на исходном датасете четыре модели AgglomerativeClustering с параметром n_clusters=3, меняя параметр linkage.

В качестве ответа укажите максимальное значение v-меры, полученное при помощи данных моделей. В задании может понадобиться или не понадобиться нормализация — это нужно проверить во время решения задания. Ответ округлите до десятых и запишите с точкой.

In [30]:
link_array = ['ward', 'complete', 'average', 'single']
for linkage in link_array:
    ac = AgglomerativeClustering(n_clusters=3, linkage=linkage)
    ac.fit(X_st)
    y_pred_ac = ac.labels_.astype(np.int)
    print(linkage, v_measure_score(labels_true=y, labels_pred=y_pred_ac))

ward 0.9079037199053294
complete 0.5805530251504777
average 0.678656551579543
single 0.0008842106330108959


### ответ 0,9 (с нормализацией)

## Задание 4.14.6

2.0/2.0 points (graded)
Сравним, как влияет предварительный расчёт матрицы смежности на качество агломеративной кластеризации.

Обучите на исходном датасете две модели AgglomerativeClustering:

с параметром n_clusters=3
с параметром n_clusters=3 и предварительно посчитанной матрицей смежности для объектов датасета
Построить матрицу смежности можно с помощью кода:

В качестве ответа введите максимальное значение v-меры, полученное при помощи данных моделей. В задании может понадобиться или не понадобиться нормализация — это нужно проверить во время решения задания. Ответ округлите до десятых и запишите с точкой.

In [32]:
from sklearn.neighbors import kneighbors_graph

connectivity = kneighbors_graph(X, n_neighbors=6, include_self=False)
connectivity = 0.5 * (connectivity + connectivity.T)




In [33]:
ac1 = AgglomerativeClustering(n_clusters=3, connectivity=None)
ac1.fit(X_st)
y_pred_ac1 = ac1.labels_.astype(np.int)
print(linkage, v_measure_score(labels_true=y, labels_pred=y_pred_ac1))

ac2 = AgglomerativeClustering(n_clusters=3, connectivity=connectivity)
ac2.fit(X_st)
y_pred_ac2 = ac2.labels_.astype(np.int)
print(linkage, v_measure_score(labels_true=y, labels_pred=y_pred_ac2))

single 0.9079037199053294
single 0.883942992495597


### ответ 0,9 (с нормализацией)

## Задание 4.14.7

2.0/2.0 points (graded)
Алгоритм DBSCAN очень требователен к параметрам: небольшое изменение в параметре eps или min_samples может изменить результат и качество кластеризации.

Обучите на исходном датасете две модели DBSCAN:

с параметрами eps=0.9, min_samples=35;
с параметрами eps=0.8, min_samples=35.
В качестве ответа укажите максимальное значение v-меры, полученное при помощи данных моделей. В задании может понадобиться или не понадобиться нормализация — это нужно проверить во время решения задания. Ответ округлите до сотых и запишите с точкой.

In [34]:
dbscan1 = DBSCAN(eps=0.9, min_samples=35)
dbscan1.fit(X)
y_pred_dbscan1 = dbscan1.labels_
print(v_measure_score(labels_true=y, labels_pred=y_pred_dbscan1))

dbscan2 = DBSCAN(eps=0.8, min_samples=35)
dbscan2.fit(X)
y_pred_dbscan2 = dbscan2.labels_
print(v_measure_score(labels_true=y, labels_pred=y_pred_dbscan2))



0.7732549110297919
0.7063100442576922


### ответ 0,9 (без нормализации)

## Задание 4.14.8

2.0/2.0 points (graded)
Особенностью алгоритма DBSCAN является то, что, помимо кластеризации, этот алгоритм определяет выбросы в выборке. Посмотрим на качество кластеризации без учёта таких объектов.

Обучите на исходном датасете модель DBSCAN с параметрами eps=0.9, min_samples=35. Посчитайте значение v-меры только для основных и граничных объектов выборки, то есть для объектов, которые не являются выбросами. Ответ округлите до сотых и запишите с точкой.

In [40]:
dbscan1 = DBSCAN(eps=0.9, min_samples=35)
dbscan1.fit(X)
y_pred_dbscan1 = dbscan1.labels_
#исключаем выбросы
y_pred_dbscan2 = []
y_excluded = []
for n, pred in enumerate(y_pred_dbscan1):
    if pred!=-1:
        y_pred_dbscan2.append(pred)
        y_excluded.append(y[n])
        
print(v_measure_score(labels_true=y_excluded, labels_pred=y_pred_dbscan2))

0.9667281300681958


### ответ 0,97 (без нормализации)

## Задание 4.14.9

2.0/2.0 points (graded)
В курсе мы рассмотрели две метода нормализации данных:

MinMax-нормализация — приведение данных к масштабу между 0 и 1.
Стандартная нормализация — данные имеют среднее 0 и стандартное отклонение 1.
Проверим, влияет ли предобработка данных на результат кластеризации. Обучите две модели AgglomerativeClustering с параметрами n_clusters=3:

на признаках исходного датасета,
предварительно трансформируйте признаки при помощи стандартной нормализации.
Посчитайте v-меру для получившихся результатов, в качестве ответа введите наибольшее значение. Ответ округлите до сотых и запишите с точкой.

In [42]:
ac1 = AgglomerativeClustering(n_clusters=3)
ac1.fit(X)
y_pred_ac1 = ac1.labels_.astype(np.int)
print(linkage, v_measure_score(labels_true=y, labels_pred=y_pred_ac1))

ac2 = AgglomerativeClustering(n_clusters=3)
ac2.fit(X_st)
y_pred_ac2 = ac2.labels_.astype(np.int)
print(linkage, v_measure_score(labels_true=y, labels_pred=y_pred_ac2))

single 0.703747024360433
single 0.9079037199053294


### ответ 0,91

## Задание 4.14.10

2.0/2.0 points (graded)
Обучите две модели AgglomerativeClustering с параметрами n_clusters=3:

на признаках исходного датасета,
предварительно трансформируйте признаки при помощи MinMax-нормализации.
Посчитайте v-меру для получившихся результатов, в качестве ответа введите наибольшее значение. Ответ округлите до сотых и запишите с точкой.



In [44]:
#стандартизация
from sklearn.preprocessing import MinMaxScaler
X_st2 = MinMaxScaler().fit_transform(X)

In [48]:
ac1 = AgglomerativeClustering(n_clusters=3)
ac1.fit(X_st2)
y_pred_ac1 = ac1.labels_.astype(np.int)
print(v_measure_score(labels_true=y, labels_pred=y_pred_ac1))

ac2 = AgglomerativeClustering(n_clusters=3)
ac2.fit(X)
y_pred_ac2 = ac2.labels_.astype(np.int)
print(v_measure_score(labels_true=y, labels_pred=y_pred_ac2))

0.8929241488344335
0.703747024360433


### ответ 0,89